In [1]:
number_of_shots = 7

In [2]:
experiment_name = "2024.05.07-06_Mixtral_7B_instruct_API_7-few-shots_SEMANTIC_SIMILARITY"

In [3]:
model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [4]:
!pip install -q openai #--progress-bar off

In [5]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [6]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [7]:
# output_datapath = "data/output/2023.11.16-1_Mistral_7B_instruct_API/"
# final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'
output_datapath = f"data/output/{experiment_name}/"
log_filename = f"logs/{experiment_name}.log"

final_report_name = 'report_sdg_classification_Mistral_7B.xlsx'

In [8]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [9]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [10]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN_2023.12.11.xlsx")

In [11]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [12]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST_2023.12.11.xlsx")
data = data.drop(columns=['Unnamed: 0'])
data.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125


In [13]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with the label only using the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

# system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

# The following are {number_of_shots} examples of expected classification given an input text:

# """+few_shots+"""

# Answer with the label only using the format:---LABEL---
# """


In [14]:
data_sel = data #.iloc[:5]

In [15]:
# Sposta il prompt dentro il ciclo

In [16]:
import time

# cnt = 0
results = []
for index, row in data_sel.iterrows():
    
    cnt = str(index)
    
    file = open(log_filename, 'a+', encoding='utf-8')
    file.write("\n\n##################################################\n")
    
    text_id = row['text_id']
    text = row["text"]
    sdg_true = row["sdg"]

    few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]    
    
    few_shots = ""

    for train_data_id in few_shots_list:

        # print("##################################################")
        # print(train_data_id)

        train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

        # print(train_row)

        # print(train_row.text.values[0])
        # print(train_row.sdg.values[0])

        input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
        output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

        few_shots += input_text
        few_shots += output_text    
    
    system_prompt = f"""In the following lines, a text enclosed in triple quotes will be given to you.

    The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

    The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

    If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

    The following are {number_of_shots} examples of expected classification given an input text:

    """+few_shots+"""

    Answer with the label only using the format:---LABEL---
    """    
    
    #query = template_part1 + text + template_part2
    query = text
    
    try:
        body = {
          "model": model_name,
          "messages": [
              {"role": "system", "content": system_prompt}, 
              {"role": "user", "content": query}],
          "temperature": 0.0
        }

        result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
        print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+str(result)+"\n")
    except Exception as e:
        result = str(e)
        print(cnt, text_id, sdg_true, "ERROR: "+result)
        file.write(str(cnt)+" "+str(text_id)+" "+str(sdg_true)+" "+"ERROR: "+result+"\n")        
    
#     print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
    
    result_extended = (cnt, text_id, sdg_true, result)
    #print(result_extended)
    
    with open(output_datapath+f'sdg_classification_{model_name.replace("/","_")}_{cnt}.pkl', 'wb') as f:
        pickle.dump(result_extended, f)
    
    results.append(result_extended)
    #cnt+=1
    
    file.close()
    
    time.sleep(0.5)
    
    

0 6668 0  ---SDG 0---
1 6099 0  ---SDG 0---  // This text is about entertainment and does not seem to relate to any specific SDG.
2 6471 0  ---SDG 16---
3 1786 0  ---SDG 0---
4 2143 0  ---SDG 7---
5 3377 0  ---SDG 0---
6 5589 0  ---SDG 0---

The text does not fall under any of the Sustainable Development Goals. It discusses the digital release of a horror game, Alan Wake 2, and related matters such as the game's focus on atmosphere and the decision to turn down a pitch for Alan Wake 2 in favor of Quantum Break. These topics are not directly related to any of the SDGs.
7 5431 0  ---SDG 7---

Explanation:

The text is about the warning issued by the Manufacturers Association of Nigeria regarding a proposed hike in electricity prices. The proposed hike in electricity prices will lead to increased production costs, decrease profit margins of manufacturers, and reduce their ability to expand operations and create new jobs. Therefore, the text is related to access to affordable and clean ene

65 4176 1  ---SDG 1---

Explanation:
The text discusses issues related to poverty measurement and redistribution systems in Colombia. It highlights problems with the Estratos classification system, which no longer aligns well with the distribution of income, and suggests solutions such as using a unique identification number for all social and tax purposes and updating the Sisben more frequently. These issues and solutions are closely related to SDG 1, which aims to end poverty in all its forms everywhere.
66 6655 1  ---SDG 1---
67 3587 1  ---SDG 1---
68 1658 1  ---SDG 1---
69 6667 1  ---SDG 1---
70 4588 1  ---SDG 2---
71 4057 1  ---SDG 1---
72 5400 1  ---SDG 1---

The text discusses poverty thresholds and their interpretation, which is directly related to SDG 1: No Poverty. The focus on poverty rates and the comparison of poverty thresholds in different countries highlights the global effort to eradicate poverty and promote well-being for all.
73 2631 1  ---SDG 1---
74 1250 1  ---SDG 

154 3039 2  ---SDG 2---
155 6419 2  ---SDG 2---

The text discusses the issue of increasing food demand due to population growth and urbanization, as well as the need for the agriculture sector to increase productivity to meet this demand. This is closely related to SDG 2, which aims to end hunger, achieve food security, improve nutrition, and promote sustainable agriculture.
156 5694 2  ---SDG 2---
(SDG 2 focuses on ending hunger, achieving food security and improved nutrition, and promoting sustainable agriculture.)
157 4433 2  ---SDG 2--- | The text describes the projected price behavior of various agricultural commodities, particularly cotton, and the factors affecting their prices. This is related to SDG 2, which aims to end hunger, achieve food security, and promote sustainable agriculture.
158 654 2  ---SDG 2---
159 4178 2  ---SDG 2---

Explanation:

The text given is about the issues and challenges in agricultural trade, specifically in relation to food security. It discusses t

254 2608 4  ---SDG 4---
255 823 4  ---SDG 4---
256 1972 4  ---SDG 4---

Explanation:
The text provided discusses the distribution of resources in education, specifically in Lithuania, and the need to consider socio-economic disadvantage in the calculation of the student basket. This topic aligns with Sustainable Development Goal (SDG) 4, which aims to ensure inclusive and equitable quality education and promote lifelong learning opportunities for all. Therefore, the appropriate label for this text is ---SDG 4---.
257 3492 4  ---SDG 4---
258 218 4  ---SDG 4---
259 3447 4  ---SDG 4---

The text is about an education reform program, which aims to improve the quality and equity of the education system in Mexico. The focus is on teacher career paths, school management, leadership, and social participation. The OECD and the Mexican Ministry of Education established an agreement to support the design and implementation of these reforms. Therefore, the text can be classified under SDG 4: Quali

356 1510 5  ---SDG 5---
357 747 5  ---SDG 5---
358 630 5  ---SDG 5---
359 5175 5  ---SDG 5---
(SDG 5 is the Sustainable Development Goal focused on Gender Equality)
360 241 6  ---SDG 6---

The text discusses water management and risks associated with it, which falls under SDG 6: Clean Water and Sanitation. The text emphasizes the importance of minimizing water risks and the role of stakeholders, which are key components of SDG 6. The text does not mention any other SDGs, so the classification is clear-cut.
361 372 6  ---SDG 6---
362 4479 6  ---SDG 6---
363 1568 6  ---SDG 6---
364 6194 6  ---SDG 6---
365 650 6  ---SDG 6---
366 4828 6  ---SDG 6---
367 3495 6  ---SDG 6---
368 2809 6  ---SDG 6---
369 4972 6  ---SDG 6---
370 3653 6  ---SDG 6--- (clean water and sanitation) as the text discusses the creation of a joint water supply and wastewater treatment company, as well as a system for monitoring water quality and pollution discharges.
371 379 6  ---SDG 6---
372 6287 6  ---SDG 6---
373 65

...

596 5361 9  ---SDG 9---

The text discusses the innovation performance of regions, specifically mentioning the early adoption of the green growth agenda and the competitive advantage in green solutions in the Nordic countries. This aligns with SDG 9, which aims to "build resilient infrastructure, promote inclusive and sustainable industrialization and foster innovation."
597 1364 9  ---SDG 9---
598 4572 9  ---SDG 9---
599 3075 9  ---SDG 9---

The text discusses the role of ICTs (Information and Communication Technologies) in enterprise development and poverty reduction, emphasizing the importance of policymakers' strategic interventions and collaboration between national governments, development partners, and other stakeholders. While it touches upon broader development objectives, the primary focus is on the integration of ICT infrastructure and connectivity with human capacity development and enterprise development, which aligns with SDG 9 (Industry, Innovation, and Infrastructure).


658 6782 10  ---SDG 10---
659 406 10  ---SDG 8---
660 1968 11  ---SDG 11---
661 1902 11  ---SDG 11---
662 6209 11  ---SDG 11---
663 2986 11  ---SDG 11---
664 3871 11  ---SDG 8---
665 1564 11  ---SDG 11---
666 4187 11  ---SDG 11---
667 4353 11  ---SDG 11---
668 4349 11  ---SDG 11---
669 3479 11  ---SDG 11---
670 1989 11  ---SDG 11---
671 739 11  ---SDG 11---
672 1381 11  ---SDG 11---
673 3418 11  ---SDG 11---
674 6481 11  ---SDG 11---

(The text is discussing the use of public-private partnerships (PPPs) in urban public investment, which aligns with SDG 11's focus on making cities inclusive, safe, resilient, and sustainable.)
675 3269 11  ---SDG 11---
676 6077 11  ---SDG 11---
677 795 11  ---SDG 11---
678 3019 11  ---SDG 11---
679 2190 11  ---SDG 11---
680 5216 11  ---SDG 11---
681 2301 11  ---SDG 11---
682 6047 11  ---SDG 11---

The text discusses the complexity of intergovernmental relations in the Gauteng city-region, focusing on policies aimed at optimizing performance and improving

769 6154 12  ---SDG 11---
770 3851 12  ---SDG 12---
771 1616 12  ---SDG 12---
772 5617 12  ---SDG 12---
773 2243 12  ---SDG 12---
(SDG 12 is the Sustainable Development Goal for Responsible Consumption and Production, which includes waste management and recycling.)
774 1134 12  ---SDG 9---

The text discusses the accreditation of organizations for issuing ISO certificates and the number of companies certified under MNS ISO 14001, which falls under SDG 9 - Industry, Innovation, and Infrastructure.
775 1049 12  ---SDG 12---
776 430 12  ---SDG 12---

The text is about promoting business models that stimulate recycling of fibers into new textile products, increasing the recycling of plastics, and minimizing the down-cycling and incineration of plastic waste. This aligns with SDG 12, which aims to ensure sustainable consumption and production patterns.
777 2354 12  ---SDG 12---
778 141 12  ---SDG 12---
779 126 12  ---SDG 12---
780 3805 13  ---SDG 13---
781 2862 13  ---SDG 13---
782 2180 13 

916 588 15  ---SDG 15---
917 2246 15  ---SDG 15---
918 1924 15  ---SDG 15---
(This goal is about protecting, restoring and promoting the sustainable use of terrestrial ecosystems, sustainably managing forests, combating desertification, halting and reversing land degradation, and halting biodiversity loss.)
919 3281 15  ---SDG 15---
920 1767 15  ---SDG 11---
921 1957 15  ---SDG 13---
922 980 15  ---SDG 15---
923 1299 15  ---SDG 15---
924 5980 15  ---SDG 15---
925 2985 15  ---SDG 15---
926 1515 15  ---SDG 15---
927 6479 15  ---SDG 15---
928 2296 15  ---SDG 15--- (Forest management and community rights to manage forests are mentioned, which aligns with SDG 15's Life on Land goal)
929 3610 15  ---SDG 15---
930 152 15  ---SDG 15---
931 658 15  ---SDG 15---

Explanation:

The text provided discusses the value of biodiversity and ecosystem services, and how they can be assessed and integrated into development and poverty reduction strategies. The focus of the text is on the natural environme

997 322 16  ---SDG 16---

Explanation:

The text discusses the relationship between human rights, ethics, and work, particularly in the context of workplace bullying and morbid forms of solidarity. This topic is closely related to SDG 16, which aims to promote peaceful and inclusive societies, provide access to justice for all, and build effective, accountable, and inclusive institutions at all levels. By addressing issues related to human rights and ethics in the workplace, the text aligns with the objectives of SDG 16.
998 6351 16  ---SDG 16---
999 4375 16  ---SDG 16---
1000 2522 16  ---SDG 5---

The article discusses women's empowerment, which falls under SDG 5: Gender Equality. The focus on power dynamics and social structures in a post-conflict setting highlights the importance of addressing gender inequality and empowering women.
1001 2889 16  ---SDG 16---
1002 3927 16  ---SDG 16---

The text discusses the importance of collaborative efforts to improve diplomacy, defense, and dev

In [17]:
files = glob.glob(output_datapath + "*.pkl")
# files

In [18]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    #print(res)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
        sdg_pred = res[3]['choices'][0]['message']['content']
    except:
        sdg_pred = res
    res4df = (res[0], res[1], res[2], sdg_pred)    
    print(res4df)
    tuple_list.append(res4df)

('1015', 4695, 16, ' ---SDG 16---\n\nExplanation:\n\nThe text discusses the role of human rights NGOs and labor unions in the struggle for democratization in Nigeria, specifically focusing on the alliances between these groups in their fight against military dictatorship. The text highlights conflicts and exchanges within civil society, which promotes social capital by mitigating hierarchically structured and asymmetrical patterns of exchange among its members. The primary goal of SDG 16 is to promote peaceful and inclusive societies for sustainable development, provide access to justice for all, and build effective, accountable, and inclusive institutions at all levels. This text aligns with SDG 16, as it discusses the importance of promoting peaceful and inclusive civil society, accountability, and effective institutions.')
('582', 714, 9, ' ---SDG 9---')
('2', 6471, 0, ' ---SDG 16---')
('782', 2180, 13, ' ---SDG 13---')
('601', 4947, 10, ' ---SDG 10---')
('144', 2727, 2, ' ---SDG 2-

In [19]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [20]:
results_processed

,cnt,id,true_label,predicted_label
0,1015,4695,16,---SDG 16---\n\nExplanation:\n\nThe text disc...
1,582,714,9,---SDG 9---
2,2,6471,0,---SDG 16---
3,782,2180,13,---SDG 13---
4,601,4947,10,---SDG 10---
...,...,...,...,...
1015,152,5700,2,---SDG 1--- (The text focuses on poverty redu...
1016,276,1547,4,---SDG 4---
1017,554,5840,9,---SDG 9---
1018,927,6479,15,---SDG 15---


In [21]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- // This text is about entertainm...
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 0---
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---\n\nExplanation:\n\nThe text disc...
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---


In [22]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- // This text is about entertainm...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16
3,3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,146,3,1786,0,---SDG 0---,0
4,4,2143,NaN,By Dipo Olowookere. The President of the Manuf...,0,NaN,NaN,NaN,124,4,2143,0,---SDG 7---,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,100,1015,4695,16,---SDG 16---\n\nExplanation:\n\nThe text disc...,16
1016,1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,60,1016,6628,16,---SDG 16---,16
1017,1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,124,1017,4563,16,---SDG 16---,16
1018,1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,109,1018,6710,16,---SDG 16---,16


In [23]:
results_processed_merged.groupby("predicted_label_clean").count()

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,,,
0,42,42,0,42,42,0,0,0,42,42,42,42,42
1,75,75,74,75,75,74,74,74,75,75,75,75,75
10,45,45,45,45,45,45,45,45,45,45,45,45,45
11,72,72,70,72,72,70,70,70,72,72,72,72,72
12,55,55,55,55,55,55,55,55,55,55,55,55,55
13,65,65,65,65,65,65,65,65,65,65,65,65,65
14,60,60,60,60,60,60,60,60,60,60,60,60,60
15,61,61,59,61,61,59,59,59,61,61,61,61,61
16,55,55,53,55,55,53,53,53,55,55,55,55,55


In [24]:
results_processed_merged.to_excel(output_datapath+final_report_name)

In [25]:
results_processed_merged = results_processed_merged[~results_processed_merged.predicted_label_clean.isna()]

In [26]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       1.00      0.70      0.82        60
           1       0.73      0.92      0.81        60
           2       0.93      0.85      0.89        60
           3       0.87      0.97      0.91        60
           4       0.92      0.95      0.93        60
           5       0.90      0.93      0.92        60
           6       1.00      0.93      0.97        60
           7       0.91      0.88      0.90        60
           8       0.79      0.73      0.76        60
           9       0.77      0.92      0.84        60
          10       0.87      0.65      0.74        60
          11       0.82      0.98      0.89        60
          12       0.96      0.88      0.92        60
          13       0.85      0.92      0.88        60
          14       0.97      0.97      0.97        60
          15       0.92      0.93      0.93        60
          16       0.96      0.88      0.92        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [27]:
# AGGIUNTO IL 2024.05.07:

In [28]:
# CORREZIONI NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI O OUTPUT NON UTILI PER DEFINIRE LA CLASSE O CLASSI NON PREVISTE 

import random

# NEL CASO IN CUI IL MODELLO RESTITUISCA DUPLICATI SELEZIONO SEMPRE IL PRIMO
results_processed_merged = results_processed_merged.drop_duplicates(subset='text_id', keep='first')

# # NEL CASO IN CUI IL MODELLO RESTITUISCA OUTPUT NON UTILI PER DEFINIRE LA CLASSE SI APPLICA LA CLASSE 0
# # results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = 0
# results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'predicted_label_clean'] = results_processed_merged.loc[results_processed_merged['predicted_label_clean'].isna(), 'sdg'] - 1

# # NEL CASO IN CUI IL MODELLO RESTITUISCA CLASSI NON PREVISTE SI APPLICA LA CLASSE 0
# results_processed_merged['predicted_label_clean'] = results_processed_merged.predicted_label_clean.apply(lambda x: 0 if int(x)>16 else x)

# Lista di valori sdg_list
sdg_list = [str(x) for x in range(17)]
# sdg_list = [x for x in range(17)]
# print(sdg_list)

# Se il valore di 'predicted_label_clean' e' NaN o un valore numerico non presente nella lista sdg_list 
# allora assegna a  'predicted_label_clean' un valore qualunque estratto a caso da sdg_list ma diverso dal valore di 'sdg' per la riga in esame
# Modifica della colonna 'predicted_label_clean' secondo le regole specificate
for index, row in results_processed_merged.iterrows():
    if pd.isna(row['predicted_label_clean']) or row['predicted_label_clean'] not in sdg_list:
#         new_value = random.choice([x for x in sdg_list if x != row['sdg']])
        results_processed_merged.at[index, 'predicted_label_clean'] = 0  # new_value

In [29]:
results_processed_merged.head(3)

,Unnamed: 0.1,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,text_length,cnt,id,true_label,predicted_label,predicted_label_clean
0,0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,38,0,6668,0,---SDG 0---,0
1,1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,15,1,6099,0,---SDG 0--- // This text is about entertainm...,0
2,2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,125,2,6471,0,---SDG 16---,16


In [30]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1020
1020
              precision    recall  f1-score   support

           0       0.93      0.70      0.80        60
           1       0.73      0.92      0.81        60
           2       0.93      0.85      0.89        60
           3       0.87      0.97      0.91        60
           4       0.92      0.95      0.93        60
           5       0.90      0.93      0.92        60
           6       1.00      0.93      0.97        60
           7       0.91      0.88      0.90        60
           8       0.79      0.73      0.76        60
           9       0.77      0.92      0.84        60
          10       0.87      0.65      0.74        60
          11       0.82      0.98      0.89        60
          12       0.96      0.88      0.92        60
          13       0.85      0.92      0.88        60
          14       0.97      0.97      0.97        60
          15       0.92      0.93      0.93        60
          16       0.96      0.88      0.92        60

    accuracy    

In [31]:
classification_report_filename = output_datapath+f'/{experiment_name}_classification_report.xlsx'
classification_report_filename

report=classification_report(y_true, y_pred, output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df[['precision', 'recall', 'f1-score']] *= 100
report_df.to_excel(classification_report_filename)

In [32]:
report_df

,precision,recall,f1-score,support
0,93.333333,70.000000,80.000000,60.000000
1,73.333333,91.666667,81.481481,60.000000
2,92.727273,85.000000,88.695652,60.000000
3,86.567164,96.666667,91.338583,60.000000
4,91.935484,95.000000,93.442623,60.000000
5,90.322581,93.333333,91.803279,60.000000
6,100.000000,93.333333,96.551724,60.000000
7,91.379310,88.333333,89.830508,60.000000
8,78.571429,73.333333,75.862069,60.000000
9,77.464789,91.666667,83.969466,60.000000


In [33]:
accuracy = report_df.loc['accuracy']['precision']
precision = report_df.loc['macro avg']['precision']
recall = report_df.loc['macro avg']['recall']
f1_score = report_df.loc['macro avg']['f1-score']

In [34]:
print(precision, recall, accuracy, f1_score)

88.82696526745782 88.23529411764706 88.23529411764706 88.13126559430206


In [35]:
print(round(precision, 1), round(recall, 1), round(accuracy, 1), round(f1_score, 1))

88.8 88.2 88.2 88.1
